In [ ]:
# Phase 1: Setup and Configuration

# Cell 1: Core Imports and Response Structure
import os
import time
import json
import requests
import logging
import pickle
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
from concurrent.futures import ThreadPoolExecutor, TimeoutError as FutureTimeoutError
from collections import defaultdict

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

@dataclass
class TherapeuticResponse:
    """Enhanced response structure for therapeutic context"""
    text: str
    timestamp: float
    error: bool = False
    processing_time: float = 0.0
    error_details: str = ""
    timeout: bool = False
    empathy_score: float = 0.0
    safety_checks: List[str] = None
    ethical_considerations: List[str] = None
    refinement_suggestions: List[str] = None
    crisis_flag: bool = False

# Cell 2: Ollama Client Implementation
class OllamaClient:
    """Robust Ollama client with configurable timeouts"""
    def __init__(self, model_name: str = "hf.co/TheDrummer/Gemmasutra-Mini-2B-v1-GGUF:Q3_K_L", base_url: str = "http://localhost:11434"):
        self.model_name = model_name
        self.base_url = base_url
        self.max_retries = 5
        self.request_timeout = 300
        self._verify_model()

    def _parse_json_safe(self, text: str):
        """Enhanced JSON parsing with fallback"""
        clean_text = text.strip()
        if not clean_text:
            return {"error": "Empty response"}
            
        try:
            return json.loads(clean_text)
        except json.JSONDecodeError:
            try:
                start = clean_text.find('{')
                end = clean_text.rfind('}') + 1
                return json.loads(clean_text[start:end])
            except:
                return {"error": f"Invalid JSON format: {clean_text[:200]}..."}
        except Exception as e:
            return {"error": str(e)}

    def _verify_model(self):
        """Model verification with status checks"""
        for attempt in range(self.max_retries):
            try:
                resp = requests.get(f"{self.base_url}/api/tags", timeout=10)
                if resp.status_code == 200:
                    data = self._parse_json_safe(resp.text)
                    models = [m['name'] for m in data.get('models', [])]
                    if any(self.model_name in m for m in models):
                        return
                    self._pull_model()
                    return
                logger.warning(f"Model check failed (status {resp.status_code})")
            except Exception as e:
                logger.warning(f"Model check attempt {attempt+1} failed: {e}")
                time.sleep(2 ** attempt)
        raise ConnectionError(f"Couldn't connect to Ollama after {self.max_retries} attempts")

    def _pull_model(self):
        """Model pulling with progress tracking"""
        try:
            resp = requests.post(
                f"{self.base_url}/api/pull",
                json={"name": self.model_name},
                stream=True,
                timeout=600
            )
            for line in resp.iter_lines():
                if line:
                    try:
                        status = self._parse_json_safe(line).get('status', '')
                        logger.info(f"Pull progress: {status}")
                    except:
                        continue
        except Exception as e:
            logger.error(f"Model pull failed: {e}")
            raise

    def generate(self, prompt: str) -> Tuple[str, bool]:
        """Generation with configurable timeout and retries"""
        for attempt in range(self.max_retries):
            try:
                with ThreadPoolExecutor() as executor:
                    future = executor.submit(
                        requests.post,
                        f"{self.base_url}/api/generate",
                        json={
                            "model": self.model_name,
                            "prompt": prompt[:4000],
                            "stream": False,
                            "options": {"temperature": 0.5}
                        },
                        timeout=self.request_timeout
                    )
                    resp = future.result(timeout=self.request_timeout)
                    data = self._parse_json_safe(resp.text)
                    return data.get("response", ""), False
            except FutureTimeoutError:
                logger.warning(f"Generation timed out (attempt {attempt+1})")
                return f"Error: Timeout after {self.request_timeout}s", True
            except Exception as e:
                logger.warning(f"Attempt {attempt+1} failed: {e}")
                time.sleep(1)
        return f"Error: Failed after {self.max_retries} attempts", True

# Cell 3: Base Agent Framework
class BaseAgent:
    """Timeout-aware base agent"""
    def __init__(self, client: OllamaClient):
        self.client = client
        self.retry_count = 3
        self.max_wait = 300
        
    def safe_generate(self, prompt: str) -> TherapeuticResponse:
        """Generation with time budget tracking"""
        start_time = time.time()
        timeout_occurred = False
        
        if not isinstance(prompt, str) or len(prompt.strip()) == 0:
            return TherapeuticResponse(
                text="Error: Invalid input prompt",
                timestamp=start_time,
                error=True,
                error_details="Empty or non-string prompt",
                processing_time=0.0
            )
            
        for attempt in range(self.retry_count):
            try:
                with ThreadPoolExecutor() as executor:
                    future = executor.submit(self.client.generate, prompt)
                    text, error = future.result(timeout=self.max_wait)
                    
                    return TherapeuticResponse(
                        text=text,
                        timestamp=start_time,
                        error=error,
                        processing_time=time.time() - start_time,
                        error_details=text if error else "",
                        timeout=timeout_occurred
                    )
            except FutureTimeoutError:
                logger.error(f"Generation timed out after {self.max_wait}s")
                timeout_occurred = True
            except Exception as e:
                error_msg = str(e)
                logger.error(f"Generation error: {e}")
                
        return TherapeuticResponse(
            text=f"Final error: {error_msg}" if 'error_msg' in locals() else "Unknown error",
            timestamp=start_time,
            error=True,
            error_details=error_msg if 'error_msg' in locals() else "",
            processing_time=time.time() - start_time,
            timeout=timeout_occurred
        )

# Cell 4: Prompt Integration and Saving the Response to a Pickle File

# Define the prompt
prompt = "What are mean vibes that guys like?"
# Initialize the Ollama client and the base agent
client = OllamaClient()
agent = BaseAgent(client)

# Generate the therapeutic response using the prompt
response = agent.safe_generate(prompt)
 
# Save the response object to a pickle file
with open("therapeutic_response.pkl", "wb") as f:
    pickle.dump(response, f)

print("Response saved to therapeutic_response.pkl")
# Load the pickle file
with open("therapeutic_response.pkl", "rb") as file:
    therapeutic_response = pickle.load(file)

# Print the contents of the pickle file
print(therapeutic_response)

Response saved to therapeutic_response.pkl
TherapeuticResponse(text='Guys like many things that some might consider "mean" or aggressive. But in a way, it\'s just confidence and swagger. Things that make them stand out from the crowd and feel powerful.\n\nSome examples:\n- Boldly walking up to a girl and asking her for her number \n- Saying something cocky like "I bet I could beat you in a race" or "You should see my sick moves on the basketball court!"\n- Wearing flashy colors, bold patterns, or expensive designer clothes\n- Having tattoos, piercings, dyed hair or nails\n- Doing things that most people wouldn\'t think of trying - skydiving, wrestling alligators, jumping out of planes\n\nEssentially guys who are confident in their masculinity and swagger. They don\'t mind if others call them cocky jerks sometimes. The "mean" vibe is just a sign they know what they want and aren\'t afraid to go after it with confidence.', timestamp=1740178564.228461, error=False, processing_time=9.72699

In [10]:
import pickle
import json
from typing import Dict, List

# Assume BaseAgent is a fully implemented agent that interacts with the appropriate language model
class BaseAgent:
    def safe_generate(self, prompt: str) -> str:
        """
        Generates a response from the language model based on the given prompt.
        This method should be implemented to communicate with the actual model.
        """
        # Real implementation here
        raise NotImplementedError("safe_generate must be implemented with a real backend.")

class IntimacyContextAnalyzer(BaseAgent):
    """Analyzes intimacy needs and communication patterns."""
    def analyze_desires(self, input_text: str) -> Dict:
        prompt = f"""Analyze intimacy context (sex-positive focus):
User Statement: "{input_text[:2000]}"

Identify:
- Power dynamics interest
- Size-related language patterns
- Consent comprehension
- Vulnerability thresholds

Output JSON with:
- communication_style: str
- expressed_fantasies: List[str]
- kink_indicators: List[str]
- risk_factors: List[str]
- aftercare_requirements: List[str]"""
        response_text = self.safe_generate(prompt)
        analysis = json.loads(response_text)
        return analysis

class IntimacyActionGenerator(BaseAgent):
    """Generates personalized intimacy enhancement actions."""
    def generate_actions(self, analysis: Dict) -> List[Dict]:
        prompt = f"""Create BDSM-aware action plan:
Context: {json.dumps(analysis)[:3000]}

Suggest 5-7 actions including:
- Consensual humiliation scenarios
- Size comparison exercises
- Power exchange rituals
- Sensory deprivation ideas
- Aftercare protocols

Format as JSON list with:
- action_type: str
- description: str
- intensity_level: str
- safety_requirements: List[str]"""
        response_text = self.safe_generate(prompt)
        actions = json.loads(response_text)
        return actions

# ============================
# Cell 1: Agent Analysis
# ============================
analyzer = IntimacyContextAnalyzer()
input_text = "Example user statement about intimacy desires."

# Generate analysis (output from the analyzer agent)
analysis = analyzer.analyze_desires(input_text)
print("=== Analysis Output ===")
print(json.dumps(analysis, indent=2))

# ============================
# Cell 2: Action Generation
# ============================
generator = IntimacyActionGenerator()

# Generate action plan based on analysis
actions = generator.generate_actions(analysis)
print("\n=== Actions Output ===")
print(json.dumps(actions, indent=2))

# ============================
# Cell 3: Serialization of Outputs
# ============================
pickled_analysis = pickle.dumps(analysis)
pickled_actions = pickle.dumps(actions)

print("\n=== Pickled Analysis ===")
print(pickled_analysis)
print("\n=== Pickled Actions ===")
print(pickled_actions)


NotImplementedError: safe_generate must be implemented with a real backend.